In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, RegressorMixin
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 
import itertools

In [2]:
dataset=pd.read_csv("iris-setosa.csv")

X = dataset.columns[1:3]
X = dataset[X]
y = dataset.columns[0]
y = dataset[y]

X = np.array(X)
y = np.array(y)
X=X.astype(float)
y=y.astype(float)
y=np.where(y==0,-1,y)

In [3]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma):
    # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

In [4]:
from cvxopt import matrix
class HYP_SVM(BaseEstimator, RegressorMixin):
    
    def __init__(self, kernel=gaussian_kernel, sigma=None, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
        self.sigma = sigma
        
    #geometric mean
    def gm(self, y_predict, y_test):
        print(y_predict)
        test_min=0
        test_max=0
        pred_min=0
        pred_max=0
        y_test=np.asarray(y_test)
        for i in range(0,len(y_test)):
            if(y_test[i]==1):
                test_min=test_min+1
            else:
                test_max=test_max+1
        #print("y_test min",test_min)       
        #print("y_test max",test_max)
        for i in range(0,len(y_predict)):
            if(y_predict[i]==1 and y_predict[i]==y_test[i]):
                pred_min=pred_min+1
            elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
                pred_max=pred_max+1
        #print("y_pred min",pred_min)       
        #print("y_pred max",pred_max)
        se=pred_min/test_min
        sp=pred_max/test_max
        #print(se,sp)
        gm=math.sqrt(se*sp)
        print("GM",gm) 
        return gm
        
    def score(self, X, y):
        y_predict=self.predict(X)
        #gm=self.gm(y_predict, y)
        correct = np.sum(y_predict == y)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        mse=mean_squared_error(y, y_predict, squared=False)
        print("Accuracy",correct/len(y_predict))
        print("Errore quadratico medio: ", mse)
        return mse
            
          
    def get_params(self, deep=True):
        return {"C": self.C, "sigma": self.sigma}        
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def m_func(self, X_train, y, decaying_func="linear"):
        n_samples, n_features = X_train.shape
        pos = len(y[y==1])
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        X_train=np.asarray(X_train)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j], self.sigma)
               # print(K[i,j])
        #print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)

        if(decaying_func=="linear"):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.8
        if(decaying_func=="exponential"):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=pos/n_samples
        r_min=1
        self.m=func[0:pos]*r_min
        #print(self.m.shape)
        self.m=np.append(self.m,func[pos:n_samples]*r_max)
        #print(self.m.shape)
        
 ##############################################################################

    #prendeva come argomento anche x_test, l'ho tolto, ho aggiunto K
    def fit(self, X_train, y, decaying_func="linear"):
        
        self.m_func(X_train, y, decaying_func)
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 

        # Gram matrix

        #print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        #print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        #print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None
            
        return self    
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv, self.sigma)
                y_predict[i] = s
                #Valori di membership?
                #print(y_predict[i]+self.b)
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X)) #per l'accuracy
        #return self.project(X) per ritornare le membership

In [5]:
C_vals = [1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3, 1e+4]
sigma = [9e-2, 9e-1, 9, 9e+1, 9e+2, 9e+3, 9e+4]
parameters = {'C': C_vals, 'sigma': sigma}
err = []
#metto shuffle a False, è richiesto nella classe di indicare la suddivisione nel training set di dati positivi e negativi
skf = StratifiedKFold(n_splits=5, shuffle=False)
for train_index, validation_index in skf.split(X, y):
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y[train_index], y[validation_index]
    
    model = HYP_SVM(C=100, kernel=gaussian_kernel, sigma=0.9)
    clf = GridSearchCV(model, parameters, cv=5)
    grid_result = clf.fit(X=X_train, y=y_train)
    best_params = clf.best_params_
    best_C = best_params['C']
    best_sigma = best_params['sigma']
    best_model = HYP_SVM(C=best_C, kernel=gaussian_kernel, sigma=best_sigma)
    print("\nBest model", best_C, best_sigma)
    best_model.fit(X_train, y_train)
    err.append(best_model.score(X_validation, y_validation))
    

     pcost       dcost       gap    pres   dres
 0: -1.0899e+01 -4.8631e+00  4e+02  2e+01  3e-16
 1: -3.4206e+00 -1.9989e+00  4e+01  2e+00  4e-16
 2: -3.8342e-01 -1.6275e+00  2e+00  4e-02  1e-15
 3: -3.1521e-01 -4.7335e-01  2e-01  4e-17  1e-15
 4: -3.1790e-01 -3.2090e-01  3e-03  3e-17  3e-16
 5: -3.1817e-01 -3.1870e-01  5e-04  1e-17  2e-16
 6: -3.1823e-01 -3.1830e-01  6e-05  5e-17  2e-16
 7: -3.1824e-01 -3.1825e-01  1e-05  3e-17  2e-16
 8: -3.1824e-01 -3.1824e-01  1e-06  1e-16  2e-16
 9: -3.1824e-01 -3.1824e-01  6e-08  3e-17  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.0899e+01 -4.8631e+00  4e+02  2e+01  3e-16
 1: -3.4206e+00 -1.9989e+00  4e+01  2e+00  4e-16
 2: -3.8342e-01 -1.6275e+00  2e+00  4e-02  1e-15
 3: -3.1521e-01 -4.7335e-01  2e-01  4e-17  1e-15
 4: -3.1790e-01 -3.2090e-01  3e-03  3e-17  3e-16
 5: -3.1817e-01 -3.1870e-01  5e-04  1e-17  2e-16
 6: -3.1823e-01 -3.1830e-01  6e-05  5e-17  2e-16
 7: -3.1824e-01 -3.1825e-01  1e-05  3e-17  2e-1

     pcost       dcost       gap    pres   dres
 0: -4.8328e+00 -3.3806e+00  5e+02  2e+01  5e-16
 1: -2.0648e+00 -2.8188e+00  2e+01  9e-01  5e-16
 2: -8.2668e-01 -1.9419e+00  2e+00  4e-02  1e-15
 3: -7.1303e-01 -8.6504e-01  2e-01  3e-16  2e-15
 4: -7.2408e-01 -7.4928e-01  3e-02  2e-16  6e-16
 5: -7.2941e-01 -7.3382e-01  4e-03  2e-16  4e-16
 6: -7.3059e-01 -7.3183e-01  1e-03  3e-16  4e-16
 7: -7.3091e-01 -7.3133e-01  4e-04  2e-17  4e-16
 8: -7.3104e-01 -7.3114e-01  1e-04  1e-16  4e-16
 9: -7.3108e-01 -7.3108e-01  3e-06  5e-17  5e-16
10: -7.3108e-01 -7.3108e-01  3e-08  5e-17  7e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.8328e+00 -3.3806e+00  5e+02  2e+01  5e-16
 1: -2.0648e+00 -2.8188e+00  2e+01  9e-01  5e-16
 2: -8.2668e-01 -1.9419e+00  2e+00  4e-02  1e-15
 3: -7.1303e-01 -8.6504e-01  2e-01  3e-16  2e-15
 4: -7.2408e-01 -7.4928e-01  3e-02  2e-16  6e-16
 5: -7.2941e-01 -7.3382e-01  4e-03  2e-16  4e-16
 6: -7.3059e-01 -7.3183e-01  1e-03  3e-16  4e-1

 2: -7.7002e-01 -1.7717e+00  1e+00  9e-17  1e-15
 3: -7.9765e-01 -8.5339e-01  6e-02  8e-17  6e-16
 4: -7.9811e-01 -8.0280e-01  5e-03  4e-17  5e-16
 5: -7.9820e-01 -8.0211e-01  4e-03  2e-16  2e-15
 6: -7.9853e-01 -7.9953e-01  1e-03  3e-16  5e-16
 7: -7.9873e-01 -7.9910e-01  4e-04  7e-17  4e-16
 8: -7.9883e-01 -7.9894e-01  1e-04  2e-17  4e-16
 9: -7.9886e-01 -7.9890e-01  4e-05  1e-16  5e-16
10: -7.9887e-01 -7.9888e-01  4e-06  1e-16  1e-15
11: -7.9888e-01 -7.9888e-01  3e-07  3e-17  8e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.5187e+01 -5.1024e+00  4e+02  2e+01  3e-15
 1: -3.1299e+00 -1.9923e+00  2e+01  8e-01  3e-15
 2: -7.7002e-01 -1.7717e+00  1e+00  9e-17  1e-15
 3: -7.9765e-01 -8.5339e-01  6e-02  8e-17  6e-16
 4: -7.9811e-01 -8.0280e-01  5e-03  4e-17  5e-16
 5: -7.9820e-01 -8.0211e-01  4e-03  2e-16  2e-15
 6: -7.9853e-01 -7.9953e-01  1e-03  3e-16  5e-16
 7: -7.9873e-01 -7.9910e-01  4e-04  7e-17  4e-16
 8: -7.9883e-01 -7.9894e-01  1e-04  2e-17  4e-

     pcost       dcost       gap    pres   dres
 0: -3.6359e+01 -2.0416e+00  6e+02  2e+01  3e-15
 1: -8.9285e-01 -2.0298e+00  7e+00  3e-01  3e-15
 2: -4.9441e-01 -1.3695e+00  1e+00  6e-03  7e-16
 3: -4.7968e-01 -4.9297e-01  1e-02  4e-05  1e-15
 4: -4.8000e-01 -4.8013e-01  1e-04  4e-07  4e-16
 5: -4.8000e-01 -4.8000e-01  2e-06  5e-09  5e-16
 6: -4.8000e-01 -4.8000e-01  2e-07  5e-10  4e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.6359e+01 -2.0416e+00  6e+02  2e+01  3e-15
 1: -8.9285e-01 -2.0298e+00  7e+00  3e-01  3e-15
 2: -4.9441e-01 -1.3695e+00  1e+00  6e-03  7e-16
 3: -4.7968e-01 -4.9297e-01  1e-02  4e-05  1e-15
 4: -4.8000e-01 -4.8013e-01  1e-04  4e-07  4e-16
 5: -4.8000e-01 -4.8000e-01  2e-06  5e-09  5e-16
 6: -4.8000e-01 -4.8000e-01  2e-07  5e-10  4e-16
Optimal solution found.
8 out of 24 predictions correct
Accuracy 0.3333333333333333
Errore quadratico medio:  1.632993161855452
     pcost       dcost       gap    pres   dres
 0: -4.7131e+01 -1

     pcost       dcost       gap    pres   dres
 0: -2.6933e+01 -2.0277e+00  6e+02  3e+01  2e-15
 1: -6.5528e-01 -2.0141e+00  9e+00  3e-01  2e-15
 2: -3.4140e-01 -1.2918e+00  1e+00  1e-02  6e-16
 3: -3.1956e-01 -3.5179e-01  3e-02  4e-05  2e-15
 4: -3.1999e-01 -3.2032e-01  3e-04  4e-07  4e-16
 5: -3.2000e-01 -3.2000e-01  3e-06  4e-09  4e-16
 6: -3.2000e-01 -3.2000e-01  3e-08  4e-11  5e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.6933e+01 -2.0277e+00  6e+02  3e+01  2e-15
 1: -6.5528e-01 -2.0141e+00  9e+00  3e-01  2e-15
 2: -3.4140e-01 -1.2918e+00  1e+00  1e-02  6e-16
 3: -3.1956e-01 -3.5179e-01  3e-02  4e-05  2e-15
 4: -3.1999e-01 -3.2032e-01  3e-04  4e-07  4e-16
 5: -3.2000e-01 -3.2000e-01  3e-06  4e-09  4e-16
 6: -3.2000e-01 -3.2000e-01  3e-08  4e-11  5e-16
Optimal solution found.
0 out of 24 predictions correct
Accuracy 0.0
Errore quadratico medio:  2.0
     pcost       dcost       gap    pres   dres
 0: -3.6360e+01 -2.0412e+00  6e+02  2e+01  3e-1

...

     pcost       dcost       gap    pres   dres
 0:  3.3400e+06 -1.2219e+09  1e+09  6e-11  2e-11
 1:  3.2939e+06 -1.9419e+07  2e+07  9e-12  3e-11
 2:  1.9732e+06 -5.6074e+06  8e+06  4e-11  3e-11
 3:  1.5286e+06 -5.0347e+06  7e+06  2e-11  2e-11
 4:  5.7885e+05 -2.7581e+06  3e+06  4e-11  2e-11
 5:  1.6647e+05 -8.5109e+05  1e+06  2e-11  2e-11
 6:  1.2477e+04 -2.2684e+05  2e+05  8e-12  1e-11
 7: -8.4561e+03 -2.0125e+05  2e+05  2e-11  9e-12
 8: -3.1744e+04 -1.4364e+05  1e+05  8e-12  7e-12
 9: -4.4045e+04 -1.2843e+05  8e+04  4e-12  7e-12
10: -5.3651e+04 -1.1041e+05  6e+04  6e-12  7e-12
11: -6.2507e+04 -8.8129e+04  3e+04  2e-12  9e-12
12: -6.8186e+04 -8.0814e+04  1e+04  5e-12  9e-12
13: -7.2198e+04 -7.4670e+04  2e+03  4e-13  1e-11
14: -7.3143e+04 -7.3321e+04  2e+02  8e-12  1e-11
15: -7.3223e+04 -7.3225e+04  2e+00  1e-11  1e-11
16: -7.3224e+04 -7.3224e+04  2e-02  9e-12  1e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  3.3400e+06 -1.2219e+09  1e+09  6e-11  2e-1

     pcost       dcost       gap    pres   dres
 0: -3.2278e+05 -1.2019e+09  1e+09  1e-11  4e-11
 1: -3.2283e+05 -1.2510e+07  1e+07  1e-11  3e-11
 2: -3.2727e+05 -6.1649e+05  3e+05  9e-11  3e-11
 3: -3.9364e+05 -5.0365e+05  1e+05  9e-13  3e-11
 4: -3.9688e+05 -5.0274e+05  1e+05  1e-10  3e-11
 5: -4.1472e+05 -5.0597e+05  9e+04  1e-11  3e-11
 6: -4.3789e+05 -4.6992e+05  3e+04  4e-11  3e-11
 7: -4.4443e+05 -4.6021e+05  2e+04  6e-11  3e-11
 8: -4.4814e+05 -4.5505e+05  7e+03  3e-11  3e-11
 9: -4.5067e+05 -4.5184e+05  1e+03  1e-11  4e-11
10: -4.5112e+05 -4.5132e+05  2e+02  5e-11  4e-11
11: -4.5121e+05 -4.5122e+05  9e+00  7e-11  4e-11
12: -4.5122e+05 -4.5122e+05  9e-02  6e-11  4e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.2278e+05 -1.2019e+09  1e+09  1e-11  4e-11
 1: -3.2283e+05 -1.2510e+07  1e+07  1e-11  3e-11
 2: -3.2727e+05 -6.1649e+05  3e+05  9e-11  3e-11
 3: -3.9364e+05 -5.0365e+05  1e+05  9e-13  3e-11
 4: -3.9688e+05 -5.0274e+05  1e+05  1e-10  3e-1

     pcost       dcost       gap    pres   dres
 0: -4.6584e+05 -6.6865e+08  7e+08  1e-10  3e-11
 1: -4.6630e+05 -7.4794e+06  7e+06  9e-11  4e-11
 2: -5.0684e+05 -8.6806e+05  4e+05  9e-11  4e-11
 3: -7.9457e+05 -8.2837e+05  3e+04  2e-10  5e-11
 4: -7.9726e+05 -8.0148e+05  4e+03  2e-10  6e-11
 5: -7.9795e+05 -7.9854e+05  6e+02  1e-10  6e-11
 6: -7.9813e+05 -7.9831e+05  2e+02  8e-11  6e-11
 7: -7.9818e+05 -7.9825e+05  8e+01  1e-10  5e-11
 8: -7.9820e+05 -7.9823e+05  3e+01  1e-10  5e-11
 9: -7.9821e+05 -7.9821e+05  3e+00  2e-10  6e-11
10: -7.9821e+05 -7.9821e+05  3e-02  2e-10  6e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.6584e+05 -6.6865e+08  7e+08  1e-10  3e-11
 1: -4.6630e+05 -7.4794e+06  7e+06  9e-11  4e-11
 2: -5.0684e+05 -8.6806e+05  4e+05  9e-11  4e-11
 3: -7.9457e+05 -8.2837e+05  3e+04  2e-10  5e-11
 4: -7.9726e+05 -8.0148e+05  4e+03  2e-10  6e-11
 5: -7.9795e+05 -7.9854e+05  6e+02  1e-10  6e-11
 6: -7.9813e+05 -7.9831e+05  2e+02  8e-11  6e-1

     pcost       dcost       gap    pres   dres
 0: -4.6671e+05 -6.6865e+08  7e+08  1e-11  3e-11
 1: -4.6717e+05 -7.4788e+06  7e+06  5e-12  4e-11
 2: -5.0804e+05 -8.6734e+05  4e+05  1e-11  4e-11
 3: -7.9706e+05 -8.2635e+05  3e+04  5e-11  6e-11
 4: -7.9994e+05 -8.0031e+05  4e+02  7e-12  7e-11
 5: -7.9998e+05 -8.0003e+05  5e+01  1e-10  6e-11
 6: -7.9999e+05 -7.9999e+05  7e+00  2e-10  7e-11
 7: -7.9999e+05 -7.9999e+05  2e+00  3e-10  6e-11
 8: -7.9999e+05 -7.9999e+05  5e-01  4e-10  6e-11
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -4.6671e+05 -6.6865e+08  7e+08  1e-11  3e-11
 1: -4.6717e+05 -7.4788e+06  7e+06  5e-12  4e-11
 2: -5.0804e+05 -8.6734e+05  4e+05  1e-11  4e-11
 3: -7.9706e+05 -8.2635e+05  3e+04  5e-11  6e-11
 4: -7.9994e+05 -8.0031e+05  4e+02  7e-12  7e-11
 5: -7.9998e+05 -8.0003e+05  5e+01  1e-10  6e-11
 6: -7.9999e+05 -7.9999e+05  7e+00  2e-10  7e-11
 7: -7.9999e+05 -7.9999e+05  2e+00  3e-10  6e-11
 8: -7.9999e+05 -7.9999e+05  5e-01  4e-10  6e-1

RuntimeError: Cannot clone object HYP_SVM(C=1, sigma=9), as the constructor either does not set or modifies parameter C

In [ ]:
print(np.mean(err))

In [ ]:
print(err)

In [ ]:
best_model.m